# Data Prep
Use the standard machine learning problem called the iris flowers
dataset. This dataset is well studied and makes a good problem for practicing on neural
networks because all four input variables are numeric and have the same scale in centimeters.
Each instance describes the properties of an observed flower’s measurements, and the output
variable is a specific iris species. The attributes for this dataset can be summarized as follows:

1. Sepal length in centimeters
2. Sepal width in centimeters
3. Petal length in centimeters
4. Petal width in centimeters
5. Class (the flower species)

This is a multiclass classification problem, meaning that there are more than two classes
to be predicted. In fact, there are three flower species. This is an important problem for
practicing with neural networks because the three class values require specialized handling.
Below is a sample of the first five of the 150 instances:

In [1]:
!head iris.data

5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa


In [3]:
# load libraries

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [5]:
# load dataset
dataframe = pd.read_csv("iris.data", header=None)
dataset = dataframe.values

X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

The output variable contains three different string values. When modeling multiclass
classification problems using neural networks, it is good practice to reshape the output attribute
from a vector of class labels to a matrix of a Boolean for each class and whether a given instance
is in that class. This is called one-hot encoding or creating dummy variables from a categorical
variable. For example, in this problem, the three class values are `Iris-setosa`, `Iris-versicolor`,
and `Iris-virginica`.

You can first encode the strings consistently to integers using the scikit-learn class LabelEncoder.
Then convert the vector of integers to a one-hot encoding using the Keras function
to_categorical().

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one-hot encoded)
dummy_y = to_categorical(encoded_Y)

# Define the Neural Network Model
The Keras library provides wrapper classes to allow you to use neural network models developed
with Keras in scikit-learn. There is a `KerasClassifier` class in SciKeras that can be used as
an estimator in scikit-learn, the base type of model in the library. The `KerasClassifier` takes
the name of a function as an argument. This function must return the constructed neural
network model, ready for training.
Below is a function that will create a baseline neural network for the iris classification
problem. It creates a simple, fully connected network with one hidden layer that contains
eight neurons. The hidden layer uses a rectifier activation function which is a good practice.
Because you used a one-hot encoding for your iris dataset, the output layer must create three
output values, one for each class. The output value with the largest value will be taken as the
class predicted by the model.

Note that a `“softmax”` activation function was used in the output layer. This ensures
the output values are in the range of 0 and 1 and may be used as predicted probabilities.
Finally, the network uses the efficient `Adam gradient descent optimization algorithm` with a
logarithmic loss function, which is called `“categorical_crossentropy”` in Keras.


In [12]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_shape=(4,), activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

You can now create your `KerasClassifier` for use in scikit-learn. You can also pass arguments
in the construction of the `KerasClassifier` class that will be passed on to the `fit()` function
internally used to train the neural network. Here, you pass the number of epochs as 200 and
batch size as 5 to use when training the model. Debugging is also turned off when training
by setting `verbose` to 0.

In [13]:
estimator = KerasClassifier(model=baseline_model, epochs=200, batch_size=5, verbose=0)

# Evaluate the Model with k-Fold Cross-Validation
You can now evaluate the neural network model on our training data. The scikit-learn library
has excellent capability to evaluate models using a suite of techniques. The gold standard
for evaluating machine learning models is k-fold cross-validation. First, define the model
evaluation procedure. Here, you set the number of folds to 10 (an excellent default) and
shuffle the data before partitioning it.

In [15]:
kfold = KFold(n_splits=10, shuffle=True)

Now, you can evaluate your model (`estimator`) on your dataset (X and dummy_y) using a 10-fold
cross-validation procedure (`KFold`). Evaluating the model only takes approximately 10 seconds
and returns an object that describes the evaluation of the ten constructed models for each of
the splits of the dataset.

In [16]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2022-09-29 22:23:21.509832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-29 22:23:21.509897: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 22:23:21.509951: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (theia-20220907-182918): /proc/driver/nvidia/version does not exist
2022-09-29 22:23:21.510547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Baseline: 98.00% (3.06%)
